In [21]:
import gym
import numpy as np
from gym import wrappers
import random as rand

env = gym.make('CartPole-v0')

In [22]:
done = False
count = 0
observation = env.reset()
print(observation)

[0.02101128 0.03122487 0.01186718 0.04932821]


In [3]:
new_weights = np.random.uniform(0,1,4)
new_weights

array([0.39069963, 0.79048461, 0.52276403, 0.92264246])

# Random weight generation method

In [20]:
best_moves = 0
episode_length = []
best_weights = np.zeros(4) #emmpty 4 digit array

for i in range (100):
    new_weights = [rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1)]
    moves_length = []
    for j in range(100):
        observation = env.reset()
        done = False
        count = 0
        while not done:
            count+=1
            action = 1 if np.dot(observation,new_weights)>0 else  0
            observation,reward,done,_ = env.step(action)
            if done:
                break
        moves_length.append(count)
    avg_moves = float(sum(moves_length)/len(moves_length))
    if avg_moves > best_moves:
        best_moves = avg_moves
        best_weights = new_weights
    episode_length.append(avg_moves)
    if i % 20 == 0:
        print("best moves is ", best_moves)
done = False
counr = 0
env = wrappers.Monitor(env, 'movefiles',force=True)
observation = env.reset()


Error: Tried to reset environment which is not done. While the monitor is active for CartPole-v0, you cannot call reset() unless the episode is over.

In [5]:
#  inputs(pos,vel,angle,vel of pole at tip)
#while not done:
            #count+=1
            #action = 1 if np.dot(observation,best_weights)>1 else  0
            #observation,reward,done,_ = env.step(action)
            #if done:
                #break

# Q Learning method

In [28]:
#converting a continuouss state into a discrete state
MAX_STATES = 10**5
GAMMA = 0.7   #discount factor
ALPHA = 0.01 #Learning rate

#q table disctionary lookup(like weights) gets state with highest reward?
def q_states_dict(q):
    max_q = float('-inf')
    for key,val in q.items():
        if val > max_q:
            max_q = val
            max_key = key
    return max_key,max_q

# our inputs
def create_bins():
    bins = np.zeros((4,10))
    bins[0] = np.linspace(-4.8,4.8,10)
    bins[1] = np.linspace(-10,10,10)
    bins[2] = np.linspace(-41.8,41.8,10)
    bins[3] = np.linspace(-10,10,10)
    return bins
#digitize bins since we are converting a continuous state to a discrete state

def assign_bins(observation,bins):
    env_state = np.zeros(4) #creates a single array to store the 4 states
    for i in range(4):
        env_state[i] = np.digitize(observation[i],bins[i])
    return env_state

def get_state_as_string(env_state):
    env_string = ''.join(str(int(e)) for e in env_state)
    return env_string

def get_all_states_string():
    env_states = []
    for i in range(MAX_STATES):
        env_states.append(str(i).zfill(4))
    return env_states

def initialise_q():
    Q = {}
    all_states = get_all_states_string()
    for state in all_states:
        Q[state] = {}
        for action in range(env.action_space.n):
            Q[state][action] = 0 #could be anything?....like a bias?...
    return Q

def play_a_game(bins,Q,eps=0.2):
    observation = env.reset()
    done = False
    count = 0
    state = get_state_as_string(assign_bins(observation,bins))
    total_reward = 0
    while not done:
        count+=1
        if rand.uniform(0,1) > eps:
            act = env.action_space.sample() #pick a random action
        else:
            act = q_states_dict(Q[state])[0]
        observation,reward,done,_ = env.step(act)
        total_reward+=reward
        if done and count < 250:
            reward = -400
        new_state = get_state_as_string(assign_bins(observation,bins))
        al,max_q_slal = q_states_dict(Q[new_state])
        Q[state][act]+=ALPHA*(reward + GAMMA*max_q_slal - Q[state][act])
        state,act = new_state, al
    return total_reward, count

def play_multiple(bins, N = 100000):
    Q = initialise_q()
    
    count=[]
    reward = []
    for n in range(N):
        epsilon = rand.uniform(0,1)
        ep_reward,ep_length = play_a_game(bins,Q,epsilon)
        if n % 200 == 0:
            print("epsilon: ",epsilon, "Reward: ", ep_reward)
        count.append(ep_length)
        reward.append(ep_reward)
    return count,reward

In [29]:
bins = create_bins()
x,y = play_multiple(bins)

epsilon:  0.38569590243111274 Reward:  16.0
epsilon:  0.8226944677984344 Reward:  11.0
epsilon:  0.3183605556214757 Reward:  17.0
epsilon:  0.40462065520630197 Reward:  22.0
epsilon:  0.14391951887976773 Reward:  32.0
epsilon:  0.06367707201052708 Reward:  19.0
epsilon:  0.055460194272512475 Reward:  9.0
epsilon:  0.006842568365254986 Reward:  17.0
epsilon:  0.38370487604195813 Reward:  13.0
epsilon:  0.4809535232030746 Reward:  19.0
epsilon:  0.9092805440404672 Reward:  12.0
epsilon:  0.08423024987185768 Reward:  49.0
epsilon:  0.493629023009272 Reward:  13.0
epsilon:  0.9688920088787898 Reward:  11.0
epsilon:  0.5662682579708423 Reward:  12.0
epsilon:  0.6008221060928318 Reward:  13.0
epsilon:  0.5391375439963987 Reward:  15.0
epsilon:  0.3090279688759783 Reward:  16.0
epsilon:  0.2864233412696531 Reward:  18.0
epsilon:  0.4219005680353457 Reward:  10.0
epsilon:  0.11973951796722815 Reward:  20.0
epsilon:  0.5219993775172032 Reward:  14.0
epsilon:  0.787383740472254 Reward:  10.0
eps

epsilon:  0.2840911008941459 Reward:  19.0
epsilon:  0.8162850506805952 Reward:  13.0
epsilon:  0.0852780773618318 Reward:  10.0
epsilon:  0.8959919397423407 Reward:  14.0
epsilon:  0.8585805557266749 Reward:  11.0
epsilon:  0.6179152070064294 Reward:  16.0
epsilon:  0.8947294109601326 Reward:  12.0
epsilon:  0.7164791888463011 Reward:  13.0
epsilon:  0.5272259353069477 Reward:  13.0
epsilon:  0.030178948272167383 Reward:  21.0
epsilon:  0.22820810726514562 Reward:  15.0
epsilon:  0.3135160591816961 Reward:  16.0
epsilon:  0.7321652648956108 Reward:  11.0
epsilon:  0.3332802302205097 Reward:  29.0
epsilon:  0.3833326766762811 Reward:  20.0
epsilon:  0.2616324758906332 Reward:  36.0
epsilon:  0.6908225596375928 Reward:  15.0
epsilon:  0.5200119817587955 Reward:  20.0
epsilon:  0.047816748438521794 Reward:  10.0
epsilon:  0.16886902588072106 Reward:  15.0
epsilon:  0.04607869788816066 Reward:  42.0
epsilon:  0.24219926276756976 Reward:  21.0
epsilon:  0.7215733395875935 Reward:  14.0
eps

KeyboardInterrupt: 

In [7]:
bins = np.zeros((4,10))
bins[0] = np.linspace(-4.8,4.8,10)
bins[1] = np.linspace(-10,10,10)
bins[2] = np.linspace(-41.8,41.8,10)
bins[3] = np.linspace(-10,10,10)

In [8]:
o = [0.1403624 , 0.38684343, 0.10561522, 0.27885742]
def assign_bins(observation,bins):
    env_state = np.zeros(4) #creates a single array to store the 4 states
    for i in range(4):
        env_state[i] = np.digitize(observation[i],bins[i])
    return env_state
def get_state_as_string(env_state):
    env_string = ''.join(str(int(e)) for e in env_state)
    return env_string
def get_all_states_string():
    env_states = []
    for i in range(MAX_STATES):
        env_states.append(str(i).zfill(4))
    return env_states

In [9]:
x = assign_bins(o,bins)

In [10]:
x_str = get_state_as_string(x)
x_str

'5555'

In [11]:
get_all_states_string()

['0000',
 '0001',
 '0002',
 '0003',
 '0004',
 '0005',
 '0006',
 '0007',
 '0008',
 '0009',
 '0010',
 '0011',
 '0012',
 '0013',
 '0014',
 '0015',
 '0016',
 '0017',
 '0018',
 '0019',
 '0020',
 '0021',
 '0022',
 '0023',
 '0024',
 '0025',
 '0026',
 '0027',
 '0028',
 '0029',
 '0030',
 '0031',
 '0032',
 '0033',
 '0034',
 '0035',
 '0036',
 '0037',
 '0038',
 '0039',
 '0040',
 '0041',
 '0042',
 '0043',
 '0044',
 '0045',
 '0046',
 '0047',
 '0048',
 '0049',
 '0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '0055',
 '0056',
 '0057',
 '0058',
 '0059',
 '0060',
 '0061',
 '0062',
 '0063',
 '0064',
 '0065',
 '0066',
 '0067',
 '0068',
 '0069',
 '0070',
 '0071',
 '0072',
 '0073',
 '0074',
 '0075',
 '0076',
 '0077',
 '0078',
 '0079',
 '0080',
 '0081',
 '0082',
 '0083',
 '0084',
 '0085',
 '0086',
 '0087',
 '0088',
 '0089',
 '0090',
 '0091',
 '0092',
 '0093',
 '0094',
 '0095',
 '0096',
 '0097',
 '0098',
 '0099',
 '0100',
 '0101',
 '0102',
 '0103',
 '0104',
 '0105',
 '0106',
 '0107',
 '0108',
 '0109',
 '0110',
 